# Finding Similar Movies

We'll start by loading up the MovieLens dataset. Using Pandas, we can very quickly load the rows of the u.data and u.item files that we care about, and merge them together so we can work with movie names instead of ID's. (In a real production job, you'd stick with ID's and worry about the names at the display layer to make things more efficient. But this lets us understand what's going on better for now.)

In [1]:
import pandas as pd
import numpy as np

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings)


In [2]:
ratings.head()
#movies.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


Now the amazing pivot_table function on a DataFrame will construct a user / movie rating matrix. Note how NaN indicates missing data - movies that specific users didn't rate.

In [3]:
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's extract a Series of users who rated Star Wars:

In [4]:
starWarsRatings = movieRatings['Aladdin (1992)']
"""tillThereRatings = movieRatings["'Til There Was You (1997)" ]
tillThereRatings.head(200)"""
userVec = ratings['user_id']
print("Number Of Users =", np.size(userVec))
#userVec.head()
starWarsRatings.head()

Number Of Users = 100003


user_id
0    NaN
1    4.0
2    NaN
3    NaN
4    NaN
Name: Aladdin (1992), dtype: float64

Pandas' corrwith function makes it really easy to compute the pairwise correlation of Star Wars' vector of user rating with every other movie! After that, we'll drop any results that have no data, and construct a new DataFrame of movies and their correlation score (similarity) to Star Wars:

In [5]:
similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,0
title,
'Til There Was You (1997),-0.034483
101 Dalmatians (1996),0.385118
12 Angry Men (1957),0.030291
187 (1997),0.609919
2 Days in the Valley (1996),0.300585
"20,000 Leagues Under the Sea (1954)",0.138365
2001: A Space Odyssey (1968),-0.067992
"39 Steps, The (1935)",0.047605
8 1/2 (1963),0.129381


(That warning is safe to ignore.) Let's sort the results by similarity score, and we should have the movies most similar to Star Wars! Except... we don't. These results make no sense at all! This is why it's important to know your data - clearly we missed something important.

In [6]:
similarMovies.sort_values(ascending=False)

title
Aladdin (1992)                                                1.000000
Turbo: A Power Rangers Movie (1997)                           1.000000
Horseman on the Roof, The (Hussard sur le toit, Le) (1995)    1.000000
8 Seconds (1994)                                              1.000000
Killer (Bulletproof Heart) (1994)                             1.000000
Bread and Chocolate (Pane e cioccolata) (1973)                1.000000
Maya Lin: A Strong Clear Vision (1994)                        1.000000
Dark City (1998)                                              1.000000
Mrs. Dalloway (1997)                                          1.000000
Nosferatu a Venezia (1986)                                    1.000000
Zeus and Roxanne (1997)                                       1.000000
Two or Three Things I Know About Her (1966)                   1.000000
Margaret's Museum (1995)                                      1.000000
Last Time I Saw Paris, The (1954)                             1.000000


Our results are probably getting messed up by movies that have only been viewed by a handful of people who also happened to like Star Wars. So we need to get rid of movies that were only watched by a few people that are producing spurious results. Let's construct a new DataFrame that counts up how many ratings exist for each movie, and also the average rating while we're at it - that could also come in handy later.

In [7]:
import numpy as np
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

Let's get rid of any movies rated by fewer than 100 people, and check the top-rated ones that are left:

In [8]:
popularMovies = movieStats['rating']['size'] >= 100
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4.283293

100 might still be too low, but these results look pretty good as far as "well rated movies that people have heard of." Let's join this data with our original set of similar movies to Star Wars:

In [9]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
101 Dalmatians (1996),109,2.908257,0.385118
12 Angry Men (1957),125,4.344000,0.030291
2001: A Space Odyssey (1968),259,3.969112,-0.067992
Absolute Power (1997),127,3.370079,-0.016733
"Abyss, The (1989)",151,3.589404,0.364484


And, sort these new results by similarity score. That's more like it!

In [11]:
df_sorted = df.sort_values(['similarity'], ascending=False)[:15]
df_sorted

,"(rating, size)","(rating, mean)",similarity
title,,,
Aladdin (1992),219,3.812785,1.000000
"Lion King, The (1994)",220,3.781818,0.514440
Cinderella (1950),129,3.581395,0.481009
I Know What You Did Last Summer (1997),100,3.060000,0.469781
Grumpier Old Men (1995),148,3.040541,0.467307
Mrs. Doubtfire (1993),192,3.411458,0.465720
Amistad (1997),124,3.854839,0.458978
That Thing You Do! (1996),176,3.465909,0.430451
Back to the Future (1985),350,3.834286,0.427098


Ideally we'd also filter out the movie we started from - of course Star Wars is 100% similar to itself. But otherwise these results aren't bad.

## Activity

100 was an arbitrarily chosen cutoff. Try different values - what effect does it have on the end results?

In [12]:
"""accept_corrs = np.arange(0.5,0.99,0.00001)
print("Correlations:", accept_corrs)
recommend = df_sorted.loc[~df_sorted['similarity'].isin(accept_corrs)]"""

recommend = df_sorted.loc[(df_sorted['similarity']>= 0.5) & (df_sorted['similarity']<=0.9999)]
print("Recommend the following:",recommend)


Recommend the following:                        (rating, size)  (rating, mean)  similarity
title                                                            
Lion King, The (1994)             220        3.781818     0.51444
